#Broyden's Method

* Given $x_0$
* Compute $B^{-1}$, where $B = JF(x_0)$
* $x = x0 - B^{-1} F(x_0)$
* for $n \geq 1$
   * Compute $v := B^{-1} (F(x) - F(x_0))$
   * Compute $s = x - x0$
   * Update $B^{-1}_{n+1} = B^{-1}_n + \frac{(s-v)s^T B^{-1}_n}{s^T v}$
   * Compute $x = x - B^{-1}_{n+1} F(x)$

In [ ]:
function broyden(F, JF, x, tol)
    k = 1
    Nmax = 1000
    xp = zeros(size(x))
    s = zeros(size(x))
    xp[:] = x[:]
    iB = inv(JF(x))
    x = x - iB * F(x)
    while norm(F(x)) > tol && k < Nmax
        v = iB * (F(x) - F(xp))
        s[:] = x[:] - xp[:]
        iB = iB + (s - v) * s' * iB / dot(s,v)
        xp[:] = x[:]
        x = x - iB * F(x)
        k = k + 1
    end
    return x
end

Test problem:
$$
\begin{cases}
3x_1 - \cos(x_2 x_3) - 0.5 &= 0 \\
x_1^2 - 81(x_2 + 0.1)^2 + \sin(x_3) + 1.06 &= 0 \\
e^{-x_1 x_2} + 20x_3 + (10\pi -3 )/3 & = 0
\end{cases},
$$
with the initial guess $x^0 = [0.1,\; 0.1,\; -0.1]^T$.  

The Jacobian matrix $J(x)$ for this system is 
$$
J(x_1, x_2, x_3) =
\begin{bmatrix}
3 & x_3 \sin(x_2 x_3) & x_2 \sin(x_2 x_3) \\
2x_1 & -162(x_2 + 0.1) & \cos(x_3) \\
-x_2 e^{-x_1 x_2} & -x_1 e^{-x_1 x_2} & 20
\end{bmatrix}
$$

In [ ]:
function F(x)
    return [ 3*x[1] - cos(x[2]*x[3]) - 1/2; x[1]*x[1] - 81*(x[2]+0.1)^2 + sin(x[3])+1.06; 
        exp(-x[1]*x[2]) + 20*x[3] + (10*pi-3)/3 ]
end

function DF(x)
    mat = [ 3 x[3]*sin(x[2]*x[3]) x[2]*sin(x[2]*x[3]); 
        2*x[1] -162*(x[2]+0.1) cos(x[3]);
        -x[2]*exp(-x[1]*x[2]) -x[1]*exp(-x[1]*x[2]) 20];
    return mat
end

In [ ]:
broyden(F, DF, [1.1;0.1;-0.1], 1e-12)
#[1;1] * [1 1]

#Bisection Method

The bisection is used for the root-finding problem, i.e. find $x$ so that $f(x) = 0$, where $f$ is a continuous function.  The main concept behind this method is the  **Intermediate Value Theorem**, which states that if $f(a)f(b) < 0$, then there exists $c \in (a,b)$ such that $f(c) = 0$.

* p = $\frac{a+b}{2}$.
    * if $f(p) = 0$, $p$ is the root
    * if $f(a)f(p) < 0$, that means the root is in $[a,p]$, and hence, set $b = p$
    * if $f(b)f(p) < 0$, that means the root is in $[p,b]$, and hence, set $a = p$
    * repeat

In [62]:
function bisection(f, a, b, tol, Nmax)
    fa = f(a)
    i = 1
    while i < Nmax
        p = a + (b - a)/2
        fp = f(p)
        if (abs(fp)) < tol
            return p
        end
        if fa * fp > 0
            a = p
            fa = fp
        else
            b = p
        end
        i = i + 1
        #println(i, '\t', p, '\t', abs(fp))
    end
end

bisection (generic function with 1 method)

In [59]:
bisection(x->x^2-1, 0, 1.5, 1e-5, 10)

2	0.75	0.4375
3	1.125	0.265625
4	0.9375	0.12109375
5	1.03125	0.0634765625
6	0.984375	0.031005859375
7	1.0078125	0.01568603515625
8	0.99609375	0.0077972412109375
9	1.001953125	0.003910064697265625
10	0.9990234375	0.0019521713256835938


# Gradient Descent Method

In [ ]:
function GradDesConst(F, JF, x, tol)
    gradG = x -> 2 * JF(x)' * F(x)
    xp = zeros(size(x))
    k = 1
    Nmax = 2000
    alpha = 1e-4
    while norm(F(x)) > tol && k < Nmax
        xp[:] = x[:]
        x = x - alpha * gradG(x)
        k = k + 1
        println(k, '\t', norm(F(x)))
    end
    return x
end

In [ ]:
GradDesConst(F, DF, [1.1;0.1;-0.1], 1e-5)

In [60]:
function GradDesBi(F, JF, x, tol, bisection)
    gradG = x -> 2 * JF(x)' * F(x)
    h = (alpha, x, d) -> dot(gradG(x-alpha*d), d)
    xp = zeros(size(x))
    k = 1
    Nmax = 2000
    alpha = 1e-4
    while norm(F(x)) > tol && k < Nmax
        xp[:] = x[:]
        alpha = bisection(a -> h(a, x, gradG(x)), 0, 1.5, 1e-5, 1000)
        x = x - alpha * gradG(x)
        k = k + 1
        println(k, '\t', norm(F(x)))
    end
    return x
end

GradDesBi (generic function with 1 method)

In [63]:
x0 = GradDesBi(F, DF, [1.1;0.1;-0.1], 1e-4, bisection)

2	1.8118948270755966
3	1.7408778349117393
4	1.5987769196164405
5	0.7789083375999117
6	0.6616792761440902
7	0.5642524892963374
8	0.47172340957830017
9	0.39606307395972584
10	0.3436397898201964
11	0.29871136815575
12	0.27635339386022606
13	0.2558005377216454
14	0.23820089648773812
15	0.22183407275613587
16	0.20675275784899672
17	0.1927009303583694
18	0.17962009471450127
19	0.16742857591596483
20	0.15606799256069862
21	0.14547946410605853
22	0.1356108090811642
23	0.12641244034166071
24	0.11783905346842238
25	0.10984782557335954
26	0.10239938188487195
27	0.09545649313041868
28	0.0889851341920356
29	0.08295273049811659
30	0.07732983800761349
31	0.07208848258761262
32	0.067202656118126
33	0.06264816652749253
34	0.05840269858180582
35	0.05444540758223863
36	0.05075623155111957
37	0.04731718071810144
38	0.04411136536585193
39	0.041122672816247964
40	0.03833684819127723
41	0.03574000934537644
42	0.033319047616563825
43	0.03106268224906487
44	0.028959422404435137
45	0.026998896977964842
46	0.025

3-element Array{Float64,1}:
  0.498171
 -0.199608
 -0.528827

0.005780205820467287
68	0.005388551238402588
69	0.00502396804377682
70	0.004685771254280579
71	0.004370435603028331
72	0.004074662901620629
73	0.003801914705352559
74	0.0035487574880814516
75	0.003307345981881306
76	0.0030809945493300286
77	0.002867183156561127
78	0.0026747805494640228
79	0.0024942119452363324
80	0.0023223457666698543
81	0.0021697447804938175
82	0.0020287557370296454
83	0.0018874875596943267
84	0.0017486649979308014
85	0.001619119172649471
86	0.0015072935946616921
87	0.0014084889254475498
88	0.0013167176005765883
89	0.001225297830021891
90	0.0011507546626820355
91	0.0010658720623060712
92	0.0009733752019669373
93	0.0008746682449998273
94	0.0007615891311178948
95	0.0006735433276614561
96	0.0005975473352864104
97	0.0005188786657964742
98	0.00028900527784583743
99	8.020854365433383e-5


In [ ]:
broyden(F, DF, x0, 1e-12)